<a href="https://colab.research.google.com/github/JotaBlanco/QuixStreamsNotebooks/blob/main/Conferences/BerlinTimeseriesMeetup/Quix_Streams_PUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Quix Streams
Just use pip install to download the Quix Streams library. 

[Quix Streams](https://github.com/quixio/quix-streams) is an open source Python library for processing streaming data. It’s aimed at people who work with time-series data streams — from developers and ML engineers to data scientists and data engineers.

In [ ]:
! pip install quixstreams

# Import the libraries
We will be using mainly pandas, quix, matplotlib and seaborn.

In [ ]:
import pandas as pd
import quixstreams as qx

# 1 - Create client
Let's start by creating a Quix client that we'll use to publish and subscribe to Kafka topics.

In [ ]:
# Initiating Quix managed token, but it could be your own kafka
token = 'sdk-296f2b9decff4770a525ff7d8855a78d'
client = qx.QuixStreamingClient(token)
client

# 2 - Producer client
To publish data to one topic, we will need to create a producer client pointing to that topic.

In [ ]:
topic_name = "test-topic"
topic_producer = client.get_topic_producer(topic_name)
topic_producer

# 3 - Streams
Streams are ways to distribute the messages load into a topic very efficiently, allowing escalation whilst ensuring chronolofical order. Streams are to topics what road lines are to highways. 
We don't need many streams yet, but let's see how they are created:

In [ ]:
stream_id = "test-stream_1"
test_stream_1 = topic_producer.create_stream(stream_id)
test_stream_1

In [ ]:
stream_id = "test-stream_2"
test_stream_2 = topic_producer.create_stream(stream_id)
test_stream_2

You can add properties to streams, like names and metadata.

In [ ]:
test_stream_1.properties.name = "Tutorial Test Stream 1"
test_stream_1.properties.metadata["Test Number"] = "1"
test_stream_2.properties.name = "Tutorial Test Stream 2"
test_stream_2.properties.metadata["Test Number"] = "2"

# 4 - Data format
There are two data formats Quix Stream can use to publish data to topics:

## 4.1 TimeseriesData
TimeseriesData is the formal class in Quix Streams which represents a time series data packet in memory. The format consists of a list of Timestamps with their corresponding parameter names and values for each timestamp.

You should imagine a TimeseriesData as a table where the Timestamp is the first column of that table and where the parameters are the columns for the values of that table.

In [ ]:
# This dataframe follows the proper TimeseriesData format: timestamp and different parameters
df = pd.DataFrame({
    "Timestamp": [pd.Timestamp.now(), pd.Timestamp.now() + pd.Timedelta("5sec")],
    "Param A": [10, 20],
    "Column B": [12, 9]
})
df

In [ ]:
# This is the actual way to define a qx.TimeseriesData object for that same data
timeseries_data = qx.TimeseriesData()
timeseries_data.add_timestamp(pd.Timestamp.now()) \
                .add_value("Param A", 10) \
                .add_value("Column B", 12)
timeseries_data.add_timestamp(pd.Timestamp.now() + pd.Timedelta("5sec")) \
                .add_value("Param A", 20) \
                .add_value("Column B", 9)

Conversions between qx.TimeseriesData and pd.Dataframe formats are easy:

In [ ]:
timeseries_data.to_dataframe()

In [ ]:
qx.TimeseriesData.from_panda_dataframe(df)

## 4.2 EventData
EventData consists of a record with a Timestamp, an EventId and an EventValue.

You should imagine a list of EventData instances as a simple table of three columns where the Timestamp is the first column of that table and the EventId and EventValue are the second and third columns, as shown in the following table.

In [ ]:
event_data_1 = qx.EventData(
    event_id = "Door Open", 
    time = pd.Timestamp.now(), 
    value = "The front door of the house has just been open")
event_data_1

In [ ]:
event_data_2 = qx.EventData(
    event_id = "Door Closed", 
    time = pd.Timestamp.now(), 
    value = "The front door of the house is back to closed state")
event_data_2

# 5 - Publish data to the topic
Let's publish each of the data messages created to one stream now:

## 5.1 TimeseriesData
Let's see how to publish the TimeseriesData object we created earlier.

In [ ]:
timeseries_data

In [ ]:
# Publishing it to stream 1 and stream 2
test_stream_1.timeseries.publish(timeseries_data)
test_stream_2.timeseries.publish(timeseries_data)

In [ ]:
# Also, pd.DataFrame objects can be published very simply  
# (conversion to qx.TimeseriesData object is done automatically)
df

In [ ]:
# Publishing it to stream 1 and stream 2
test_stream_1.timeseries.publish(df)
test_stream_2.timeseries.publish(df)

In [ ]:
test_stream_1.timeseries.publish(df)
print("Sent")

## 5.2 EventData
Let's now publish the EventData messages we created earlier:

In [ ]:
# Publishing event 1 to stream 1, and event 2 to stream 2
test_stream_1.events.publish(event_data_1)
test_stream_2.events.publish(event_data_2)

##5.3 - Publish random data

In [ ]:
import random

In [ ]:
df = pd.DataFrame({
    "Timestamp": [pd.Timestamp.now()],
    "Param A": [random.randint(10, 20)],
    "Param B": [random.randint(0, 10)]
})
test_stream_1.timeseries.publish(df)